# Sample t5_base

## Import libraries

In [1]:
from pyspark.ml import Pipeline

import sparknlp
from sparknlp.annotator import T5Transformer
from sparknlp.base import DocumentAssembler, LightPipeline

## Start Spark

In [2]:
spark = sparknlp.start()
print("Spark NLP Version :", sparknlp.version())

25/03/06 23:41:15 WARN Utils: Your hostname, ductrung-ASUS-TUF-Gaming-F15-FX507ZC-FX507ZC resolves to a loopback address: 127.0.1.1; using 192.168.118.101 instead (on interface wlo1)
25/03/06 23:41:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ductrung/spark-3.5.3-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ductrung/.ivy2/cache
The jars for the packages stored in: /home/ductrung/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f30eef6f-eeb5-4f70-8dda-4738f9442782;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in ce

Spark NLP Version : 5.5.3


## Document Assambler

In [3]:
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("documents")

## Model

In [4]:
t5 = (
    T5Transformer()
    .pretrained("t5_base", "en")
    .setTask("summarize:")
    .setMaxOutputLength(70)
    .setInputCols(["documents"])
    .setOutputCol("summaries")
)

t5_base download started this may take some time.
Approximate size to download 451.8 MB
[ | ]

25/03/06 23:41:32 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/03/06 23:41:32 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


t5_base download started this may take some time.
Approximate size to download 451.8 MB
Download done! Loading the resource.
[ \ ]

2025-03-06 23:41:41.229147: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-06 23:41:41.397310: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98697216 exceeds 10% of free system memory.


[ | ]

2025-03-06 23:41:41.699629: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98697216 exceeds 10% of free system memory.
2025-03-06 23:41:42.610398: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98697216 exceeds 10% of free system memory.
2025-03-06 23:41:42.641013: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98697216 exceeds 10% of free system memory.
2025-03-06 23:41:43.073218: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98697216 exceeds 10% of free system memory.


[ | ]

[OK!]


## Pipeline

In [5]:
summarizer_pp = Pipeline(stages=[document_assembler, t5])
empty_df = spark.createDataFrame([[""]]).toDF("text")
pipeline_model = summarizer_pp.fit(empty_df)
sum_lmodel = LightPipeline(pipeline_model)

## Input data

In [6]:
example_txt = """
Transfer learning, where a model is first pre-trained on a data-rich task before being finetuned on a downstream task, has emerged as a powerful technique in natural language processing (NLP). The effectiveness of transfer learning has given rise to a diversity of approaches, methodology, and practice. In this paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework that converts all text-based language problems into a text-to-text format. Our systematic study compares pre-training objectives, architectures, unlabeled data sets, transfer approaches, and other factors on dozens of language understanding tasks. By combining the insights from our exploration with scale and our new “Colossal Clean Crawled Corpus”, we achieve state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more. To facilitate future work on transfer learning for NLP, we release our data set, pre-trained models, and code.
"""

## Output

In [7]:
res = sum_lmodel.fullAnnotate(example_txt)[0]
print("Summary:", res["summaries"][0].result)

Summary: transfer learning is a technique in natural language processing (NLP) the technique is used to fine tune a model for a downstream task . the paper explores the landscape of transfer learning techniques for NLP .
